In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
import sinr.graph_embeddings as ge
import sinr.text.evaluate as ev
#Subsample of OANC corpus as a co-occurrence matrix
sinr = ge.SINr.load_from_cooc_pkl("oanc_extracted_min_freq20_max_freq5900_prct_sampling1_matrix.pkl")

/lium/home/salhallak/miniconda3/envs/sinr_dev/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-07-22 13:49:36,344 - load_from_cooc_pkl - INFO - Building Graph.
2025-07-22 13:49:36,346 - load_pkl_text - INFO - Loading cooccurrence matrix and dictionary.
2025-07-22 13:49:36,409 - load_pkl_text - INFO - Finished loading data.
2025-07-22 13:49:38,819 - load_from_cooc_pkl - INFO - Finished building graph.


In [3]:
len(sinr.wrd_to_idx), "words of vocabulary in this subsample"

(17706, 'words of vocabulary in this subsample')

In [2]:
sinr.detect_communities(gamma=80)

2025-07-22 13:49:46,374 - detect_communities - INFO - Detecting communities.


Gamma for louvain : 80


2025-07-22 13:49:47,503 - detect_communities - INFO - Finished detecting communities.


Communities detected in 1.08262 [s]
solution properties:
-------------------  ------------
# communities        4904
min community size      1
max community size     32
avg. community size     3.61052
modularity              0.0345829
-------------------  ------------


In [3]:
sinr.extract_embeddings()

2025-07-22 13:49:50,128 - extract_embeddings - INFO - Extracting embeddings.
2025-07-22 13:49:50,131 - extract_embeddings - INFO - Applying NFM.
2025-07-22 13:49:50,136 - get_nfm_embeddings - INFO - Starting NFM
2025-07-22 13:50:06,626 - extract_embeddings - INFO - NFM successfully applied.
2025-07-22 13:50:06,628 - extract_embeddings - INFO - Finished extracting embeddings.


In [4]:
modele = ge.InterpretableWordsModelBuilder(sinr, "modele", n_jobs=8, n_neighbors=4).build()
ev.similarity_MEN_WS353_SCWS(modele)

94 missing words


27 missing words


235 missing words


{'MEN': 0.4606079047763926,
 'WS353': 0.4635494913331757,
 'SCWS': 0.42875525129769765}

In [7]:
modele.vectors.shape, "(nb of words, dimensions)"

((17706, 4920), '(nb of words, dimensions)')

In [8]:
#Small Subsample of OANC corpus as a co-occurrence matrix
sinr_small = ge.SINr.load_from_cooc_pkl("oanc_extracted_min_freq50_max_freq3000_prct_sampling08_matrix.pkl")

2025-07-03 09:45:48,415 - load_from_cooc_pkl - INFO - Building Graph.
2025-07-03 09:45:48,415 - load_pkl_text - INFO - Loading cooccurrence matrix and dictionary.
2025-07-03 09:45:48,425 - load_pkl_text - INFO - Finished loading data.
2025-07-03 09:45:49,086 - load_from_cooc_pkl - INFO - Finished building graph.


In [9]:
len(sinr_small.wrd_to_idx), "words of vocabulary"

(9494, 'words of vocabulary')

In [10]:
len(sinr.wrd_to_idx) - len(sinr_small.wrd_to_idx), "words are present in the big subsample, but absent in the small one"

(8212, 'words are present in the big subsample, but absent in the small one')

In [11]:
#Transferring the communities of the bigger model to the small one and align the models
modele_small_bigasaref = modele.transfert_sinr(sinr_small)

2025-07-03 09:46:12,656 - extract_embeddings - INFO - Extracting embeddings.
2025-07-03 09:46:12,657 - extract_embeddings - INFO - Applying NFM.
2025-07-03 09:46:12,658 - get_nfm_embeddings - INFO - Starting NFM
2025-07-03 09:46:17,558 - extract_embeddings - INFO - NFM successfully applied.
2025-07-03 09:46:17,558 - extract_embeddings - INFO - Finished extracting embeddings.


### Interpreting the small model in the big one's referential

The small model and its projected version into the big one's referential have the same neighborings, but their neighboring may be different from the big one.

In [12]:
"Big modele : ", modele.vectors.shape, "(nb of words, dimensions)"

('Big modele : ', (17706, 4920), '(nb of words, dimensions)')

In [13]:
"Transferred modele : ", modele_small_bigasaref.vectors.shape, "(nb of words, dimensions)"

('Transferred modele : ', (9494, 4920), '(nb of words, dimensions)')

In [14]:
modele_small_bigasaref.most_similar("cheese")

{'object ': 'cheese',
 'neighbors ': [('bread', 0.37), ('wine', 0.35), ('bottle', 0.32)]}

In [15]:
modele.most_similar("cheese")

{'object ': 'cheese',
 'neighbors ': [('foodstuff', 0.5), ('sausage', 0.47), ('bread', 0.41)]}

The small model and its projected version into the big one's referential have the same stereotypes, but the dimensions ids are different. The dimensions ids of the projected versions are aligned with the big model.

In [16]:
modele.get_obj_stereotypes("hippocampus")

[{'dimension': 769,
  'value': True,
  'stereotypes': [(0.22, 'hippocampus'),
   (0.21, 'furrow'),
   (0.19, 'amygdala')]},
 {'dimension': 1966,
  'value': True,
  'stereotypes': [(0.38, 'tclr'), (0.1, 'spleen'), (0.09, 'intestine')]},
 {'dimension': 2525,
  'value': True,
  'stereotypes': [(0.33, 'cord'), (0.32, 'l6-s1'), (0.29, 'spinal')]},
 {'dimension': 1201,
  'value': True,
  'stereotypes': [(0.1, 'autistic'), (0.06, 'amyloid'), (0.06, 'amygdala')]},
 {'dimension': 297,
  'value': True,
  'stereotypes': [(0.47, 'rdta'), (0.36, 'scid'), (0.3, 'balb')]}]

In [17]:
modele_small_bigasaref.get_obj_stereotypes("hippocampus")

[{'dimension': 769,
  'value': True,
  'stereotypes': [(0.26, 'furrow'), (0.22, 'hippocampus'), (0.2, 'cortex')]},
 {'dimension': 1966,
  'value': True,
  'stereotypes': [(0.11, 'cirrhosis'), (0.11, 'intestine'), (0.09, 'spleen')]},
 {'dimension': 3431,
  'value': True,
  'stereotypes': [(0.03, 'hippocampus'), (0.03, 'lifelong'), (0.03, 'diet')]},
 {'dimension': 2525,
  'value': True,
  'stereotypes': [(0.35, 'cord'), (0.29, 'spinal'), (0.28, 'dorsal')]},
 {'dimension': 3260,
  'value': True,
  'stereotypes': [(0.15, 'pheromone'), (0.12, 'female'), (0.12, 'males')]}]

In [18]:
modele.get_obj_stereotypes("beverage")

[{'dimension': 642,
  'value': True,
  'stereotypes': [(0.12, 'caloric'), (0.11, 'beverage'), (0.08, 'intake')]},
 {'dimension': 643,
  'value': True,
  'stereotypes': [(0.1, 'beverage'), (0.08, 'alcoholic'), (0.08, 'foc')]},
 {'dimension': 4765,
  'value': True,
  'stereotypes': [(0.18, 'overdominance'),
   (0.16, 'k1'),
   (0.11, 'phosphatidylinositol')]},
 {'dimension': 712,
  'value': True,
  'stereotypes': [(0.1, 'bottled'), (0.05, 'gin'), (0.04, 'vodka')]},
 {'dimension': 444,
  'value': True,
  'stereotypes': [(0.17, 'parenchyma'),
   (0.15, 'extravascular'),
   (0.13, 'artifactual')]}]

In [19]:
modele_small_bigasaref.get_obj_stereotypes("beverage")

[{'dimension': 642,
  'value': True,
  'stereotypes': [(0.14, 'beverage'), (0.09, 'intake'), (0.06, 'calorie')]},
 {'dimension': 643,
  'value': True,
  'stereotypes': [(0.11, 'beverage'), (0.07, 'alcoholic'), (0.02, 'citrus')]},
 {'dimension': 712,
  'value': True,
  'stereotypes': [(0.04, 'alcoholic'), (0.03, 'beverage'), (0.02, 'liquor')]},
 {'dimension': 1617,
  'value': True,
  'stereotypes': [(0.05, 'antioxidant'), (0.04, 'fortify'), (0.04, 'bottle')]},
 {'dimension': 444,
  'value': True,
  'stereotypes': [(0.07, 'adherent'),
   (0.07, 'emphysema'),
   (0.06, 'neutrophil')]}]

# OOV Word Transfer

#### Creating a mini corpus with words that are out-of-vocabulary (OOV) relative to the OANC


In [5]:
mini_corpus = [
    "tiktok is a social media platform",
    "many teenagers use tiktok and snapchat daily",
    "tiktok trends spread fast",
    "chatgpt helps users with creative tasks",
    "midjourney generates stunning ai art",
    "biobert outperformed traditional embeddings",
    "openai released gpt-4 in 2023",
    "stable diffusion can generate images from prompts",
    "llama2 is another popular open-source model",
    "huggingface hosts thousands of transformers",
    "twitter was rebranded to x by elon musk",
    "kpbionlp is a specialized biomedical corpus",
    "biosses is used for sentence similarity tasks",
    "peft techniques improve model fine-tuning"
]


In [6]:
import re

# Extract the vocabulary from the mini_corpus
def extract_vocabulary_from_corpus(corpus):
    words = []
    for line in corpus:
        tokens = [w for w in re.findall(r'\b[\w\-]+\b', line.lower()) if w.isascii() and len(w) >= 2]
        words.extend(tokens)
    return set(words)

corpus_vocab = extract_vocabulary_from_corpus(mini_corpus)



In [7]:
# Filter out stopwords to avoid treating them as OOVs
def filter_stopwords(words):
    STOPWORDS = {
        'a','an','the','and','for','are','but','not','you','all','can','had','her','was','one','our','out','day','get',
        'has','him','his','how','man','new','now','old','see','two','way','who','boy','did','its','let','put','say','she',
        'too','use','into','than','time','very','when','much','then','them','well','were','been','good','just','long',
        'made','many','over','such','take','come','could','would','should','will','shall','may','might','must','first',
        'never','think','where','being','every','these','those','like','back','only','know','little','year','years',
        'more','between','during','before','after','above','below','down','there','here','this','that','with','from',
        'they','have','what','which','about','also','other','some','same','each','in','on','at','as','is','of','to','by',
        'be','if','do','does','done','so','no','yes','yet','any','each','than','ever','still','because','shown','either','however'
    }
    return {w for w in words if w not in STOPWORDS and len(w) > 2}



In [8]:
# Detect OOV words (present in the mini_corpus but not in the oanc model)
def detect_oov_words(sinr_cooc, vocabulary):
    filtered = filter_stopwords(vocabulary)
    model_vocab = set(sinr_cooc.wrd_to_idx)
    oov = filtered - model_vocab
    print(f"Filtered corpus words: {len(filtered)}")
    print(f"OOV detected: {len(oov)} ({len(oov)/len(filtered)*100:.2f}%)")
    return oov

oov_words = detect_oov_words(sinr, corpus_vocab)


Filtered corpus words: 57
OOV detected: 34 (59.65%)


In [9]:
print("First 10 OOV words:", list(oov_words)[:10])

First 10 OOV words: ['open-source', 'kpbionlp', 'snapchat', 'tasks', 'gpt-4', 'rebranded', 'huggingface', 'fine-tuning', 'hosts', 'techniques']


In [10]:
#  Add the OOV words 
new_ids = sinr.add_oov_words(oov_words)

#  Add edges for those OOVs using a context window 
sinr.add_edges_from_sentences(mini_corpus, new_ids, window_size=10)


34 OOV words added.
Adding edges for OOVs from sentences (window=10)...


Scanning sentences: 100%|██████████| 14/14 [00:00<00:00, 13434.06it/s]

89 edges found, 89 after filtering (threshold=1)
89 edges added to the graph.


In [11]:
updated_model = modele.transfert_sinr(sinr)


2025-07-22 13:50:59,846 - extract_embeddings - INFO - Extracting embeddings.
2025-07-22 13:50:59,849 - extract_embeddings - INFO - Applying NFM.
2025-07-22 13:50:59,852 - get_nfm_embeddings - INFO - Starting NFM
2025-07-22 13:51:16,067 - extract_embeddings - INFO - NFM successfully applied.
2025-07-22 13:51:16,069 - extract_embeddings - INFO - Finished extracting embeddings.


In [12]:
# Save the updated model
model_path = f"oanc_oov.pk"
updated_model.save(model_path)
print(f"Enriched model saved at: {model_path}")


Enriched model saved at: oanc_oov.pk


In [14]:
updated_model.most_similar("tiktok")


{'object ': 'tiktok',
 'neighbors ': [('trends', 0.58), ('snapchat', 0.41), ('teenagers', 0.41)]}